In [1]:
import pandas as pd
import numpy as np

import datetime as dt

import matplotlib.pylab as plt

from scipy import interpolate
from scipy.stats import norm
from scipy.optimize import brentq
from scipy.integrate import quad

### 1. Black-Scholes Model (Geometric - Spot price process)


In [2]:
def BlackScholesCall(S, K, r, sigma, T):
    d1 = (np.log(S/K)+(r+sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    return S*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)
def BlackScholesPut(S, K, r, sigma, T):
    d1 = (np.log(S/K)+(r+sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    return K*np.exp(-r*T)*norm.cdf(-d2) - S*norm.cdf(-d1)

### 2. Bachelier model (Arithmetic process) 

In [3]:
# Vanilla call/put
def BachelierCall(S, K, sigma,r, T):
    c = (S-K) / (sigma*S*np.sqrt(T)) # payoff is non-zero when S0 + σ√T x − K > 0
    return np.exp(-r*T)*((S-K)*norm.cdf(c) + sigma*S*np.sqrt(T)*norm.pdf(c))

def BachelierPut(S, K, sigma,r, T):
    c = (S-K) / (sigma*S*np.sqrt(T))
    return np.exp(-r*T)*((K-S)*norm.cdf(-c) + sigma*S*np.sqrt(T)*norm.pdf(-c))

# Project — Part III (Static Replication)

Project — Part III (Static Replication)

Suppose on 1-Dec-2020, we need to evaluate an exotic European derivative expiring on 15-Jan-2021 which pays:
- Payoff function:
S1/3T + 1.5 × log(ST ) + 10.0
- “Model-free” integrated variance:
$σ2MFT = E[∫ T0σ2t dt]$

Determine the price of these 2 derivative contracts if we use:

- Black-Scholes model (what σ should we use?)

- Bachelier model (what σ should we use?)

- Static-replication of European payoff (using the SABR model calibrated in the previous question)

For any twice-differentiable payoff $h(S_T)$, Breeden-Litzenberger states that

\begin{equation*}
    \begin{split}
      V_0 = e^{-rT}h(F) + \underbrace{\int_0^{F}h''(K)P(K)\;dK}_{\mbox{put integral}} + \underbrace{\int_{F}^{\infty}h''(K)C(K)\;dK}_{\mbox{call integral}}\\
    \end{split}
\end{equation*}

  
To implement the Carr-Madan static replication formula in Python, we need to be able to handle numerical integration for the put/call integrals.

# _Import Data_

In [4]:
# Discount Rate
rate_df= pd.read_csv('zero_rates_20201201.csv')

# SPX General Data
spx_df = pd.read_csv('SPX_options.csv')
spx_df['strike_price'] = spx_df['strike_price']/1000
spx_df['mid_price'] = (spx_df['best_bid'] + spx_df['best_offer'])/2

# SPX Maturity Data
spx = spx_df[(spx_df.exdate == 20210115)]


# SPX Call
spx_C1 = spx_df[(spx_df.exdate == 20201218) & (spx_df.cp_flag == 'C')]
spx_C2 = spx_df[(spx_df.exdate == 20210115) & (spx_df.cp_flag == 'C')]

# SPX Put
spx_P2 = spx_df[(spx_df.exdate == 20210115) & (spx_df.cp_flag == 'P')]

#Parameters
n = len(spx_C1.index)
strike = spx_C1['strike_price'].values

# Time To Maturity
today = dt.date(2020, 12, 1)
exdate_2 = dt.date(2021, 1, 15)
T = (exdate_2-today).days/365.0
x = rate_df['days']
y = rate_df['rate']
f = interpolate.interp1d(x,y)
r = f(T*365)/100
S = 3662.45
K = 3660
F2 = np.exp(r*T)*S

## _Implied Volatility_

In [5]:
# Implied European Options Volatility Model
def impliedCallVolatility(S, K, r, price, T):
    try:
        impliedVol = brentq(lambda x: price -
                        BlackScholesCall(S, K, r, x, T),
                        1e-6, 1)
    except Exception:
        impliedVol = np.nan
 
    return impliedVol

def impliedPutVolatility(S, K, r, price, T):
    try:
        impliedVol = brentq(lambda x: price -
                        BlackScholesPut(S, K, r, x, T),
                        1e-6, 1)
    except Exception:
        impliedVol = np.nan

    return impliedVol

In [6]:
K2 = F2

atm_call2 = 92.9242503798152
atm_put2 = 96.53426224045961

sigma_call2 = impliedCallVolatility(S, K2, r, atm_call2, T)
sigma_put2 = impliedPutVolatility(S, K2, r, atm_put2, T)
sigma2 = (sigma_call2 + sigma_put2)/2

In [7]:
sigma2

0.18467950462858124

# _Black Scholes Model Integrated Variance_

In [8]:
# Black Scholes Model Integrated Variance Function
def callintegrand(K, S, r, T, sigma2):
    price = BlackScholesCall(S, K, r, sigma2, T) / K**2
    return price

def putintegrand(K, S, r, T, sigma2):
    price = BlackScholesPut(S, K, r, sigma2, T) / K**2
    return price

In [9]:
F = S * np.exp(r*T)
I_put = quad(lambda x: putintegrand(x, S, r, T, sigma2), 0.0, F)
I_call = quad(lambda x: callintegrand(x, S, r, T, sigma2), F, 5000)
E_var_BSM = 2*np.exp(r*T)*(I_put[0] + I_call[0])

print('Black Scholes Model Integrated Variance: %.9f' % E_var_BSM)


Black Scholes Model Integrated Variance: 0.004204913


# _Black Scholes Model Derivative Pricing_

In [10]:
# Black Scholes Model Derivative Pricing Model
sigma_BSM = np.sqrt(E_var_BSM/T)

def payoff_bsm(S,r,T,sigma2):
    payoff = S ** (1/3) * np.exp((1/3) * r * T - (1/9) *sigma2 ** 2 * T) + 1.5 * (np.log(S) + r * T - 0.5 * (sigma2 ** 2) * T) + 10
    return payoff

payoff_bs = payoff_bsm(S,r,T,sigma_BSM)

print('Black-Scholes Model Derivating Pricing: %.9f' % payoff_bs)

Black-Scholes Model Derivating Pricing: 37.714459030


# _Bachelier Model Integrated Variance_

In [11]:
# Bachelier Model Integrated Variance Function
def callintegrandb(K, S, r, T, sigma2):
    price = BachelierCall(S, K,r, sigma2, T) / K**2
    return price

def putintegrandb(K, S, r, T, sigma2):
    price = BachelierPut(S, K,r, sigma2, T) / K**2
    return price

In [12]:
F = S * np.exp(r*T)
I_put = quad(lambda x: putintegrandb(x, S, r, T, sigma2), 0.0, F)
I_call = quad(lambda x: callintegrandb(x, S, r, T, sigma2), F, 5000)
E_var_B = 2*np.exp(r*T)*(I_put[0] + I_call[0])

print('Bachelier Model Integrated Variance: %.9f' % E_var_B)

Bachelier Model Integrated Variance: 0.000000570


# _Bachelier Model Derivative Pricing_

In [13]:
# Bachelier Model Derivative Pricing Model
sigma_B = np.sqrt(E_var_B/T)

x_bachelier = lambda x: (((S + sigma_B*np.sqrt(T)*x)**(1/3))+ (1.5* np.log(S + sigma_B*np.sqrt(T)*x)) +10 )*np.exp(-(x**2/2))
x_b,err = quad(x_bachelier, -(np.inf), np.inf)

derivative_bachelier = np.exp(-r*T)/np.sqrt(2*np.pi)*x_b

print('Bachelier Model Derivative Pricing: %.9f' % derivative_bachelier)

Bachelier Model Derivative Pricing: 37.713596818


We can test our static replication implementation with

\begin{equation*}
\begin{split}
    \mathbb{E}\left[\int_0^T\sigma_t^2 \;dt\right] = 2e^{rT} \left(\int_0^{F}\frac{P(K)}{K^2}\;dK + \int_{F}^{\infty}\frac{C(K)}{K^2}\;dK\right)
    \end{split}
\end{equation*}

Suppose we assume that market is following Black76 model, then we have

# _SABR Model_

In [14]:
def SABR(F, K, T, alpha, beta, rho, nu):
    X = K
    # if K is at-the-money-forward
    if abs(F - K) < 1e-12:
        numer1 = (((1 - beta)**2)/24)*alpha*alpha/(F**(2 - 2*beta))
        numer2 = 0.25*rho*beta*nu*alpha/(F**(1 - beta))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        VolAtm = alpha*(1 + (numer1 + numer2 + numer3)*T)/(F**(1-beta))
        sabrsigma = VolAtm
    else:
        z = (nu/alpha)*((F*X)**(0.5*(1-beta)))*np.log(F/X)
        zhi = np.log((((1 - 2*rho*z + z*z)**0.5) + z - rho)/(1 - rho))
        numer1 = (((1 - beta)**2)/24)*((alpha*alpha)/((F*X)**(1 - beta)))
        numer2 = 0.25*rho*beta*nu*alpha/((F*X)**((1 - beta)/2))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        numer = alpha*(1 + (numer1 + numer2 + numer3)*T)*z
        denom1 = ((1 - beta)**2/24)*(np.log(F/X))**2
        denom2 = (((1 - beta)**4)/1920)*((np.log(F/X))**4)
        denom = ((F*X)**((1 - beta)/2))*(1 + denom1 + denom2)*zhi
        sabrsigma = numer/denom

    return sabrsigma

In [15]:
def SABRCall(S, K, r, alpha, beta, rho, nu, T):
    sabr_vol = SABR(S*np.exp(r*T), K, T, alpha, beta, rho, nu)
    return BlackScholesCall(S, K, r, sabr_vol, T)


def SABRPut(S, K, r, alpha, beta, rho, nu, T):
    sabr_vol = SABR(S*np.exp(r*T), K, T, alpha, beta, rho, nu)
    return BlackScholesPut(S, K, r, sabr_vol, T)


def sabrcallintegrand(K, S, r, T, alpha, beta, rho, nu):
    price = SABRCall(S, K, r, alpha, beta, rho, nu, T) / K**2
    return price


def sabrputintegrand(K, S, r, T, alpha, beta, rho, nu):
    price = SABRPut(S, K, r, alpha, beta, rho, nu, T) / K**2
    return price


# _SABR Model Integrated Variance_

In [16]:
alpha = 1.212
beta = 0.7
rho = -0.301
nu = 5.460

F = S * np.exp(r*T)
I_put = quad(lambda x: sabrputintegrand(x, S, r, T, alpha, beta, rho, nu), 1e-6, F)
I_call = quad(lambda x: sabrcallintegrand(x, S, r, T, alpha, beta, rho, nu), F, 5000)
E_var_SABR = 2*np.exp(r*T)*(I_put[0] + I_call[0])

print('SABR Model Integrated Variance: %.9f' % E_var_SABR)

SABR Model Integrated Variance: 0.999860928


# _SABR Model Derivative Pricing Model_

In [17]:
# SABR Model Derivative Pricing Model
sigma_SABR = np.sqrt(E_var_SABR/T)

x_SABR_bachelier = lambda x: (((S + sigma_SABR*np.sqrt(T)*x)**(1/3))+ (1.5* np.log(S + sigma_SABR*np.sqrt(T)*x)) +10 )*np.exp(-(x**2/2))
x_SABR,err = quad(x_SABR_bachelier, -(np.inf), np.inf)

derivative_SABR_bachelier = np.exp(-r*T)/np.sqrt(2*np.pi)*x_SABR

print('Bachelier Model Derivative Pricing (SABR): %.9f' % derivative_SABR_bachelier)

payoff_SABR_bs = payoff_bsm(S,r,T,sigma_SABR)

print('Black-Scholes Model Derivative Pricing (SABR): %.9f' % payoff_SABR_bs)

Bachelier Model Derivative Pricing (SABR): 37.713596634
Black-Scholes Model Derivative Pricing (SABR): 35.354015550
